In [ ]:
! pip install --upgrade numpy
! pip install --upgrade pandas
! pip install --upgrade pandas-datareader

In [16]:
import pandas_datareader as dados
import pandas as pd

#ativo = "AMZN"  # https://finance.yahoo.com/ ##Amazon
#ativo = "NTCO3.SA"  # https://finance.yahoo.com/ ##Natura
#ativo = "VALE3.SA"  # https://finance.yahoo.com/ ##Vale

ativo = "AMZN"  # https://finance.yahoo.com/

##Dados_Ativo = os dados biaxados da api iniciando e finalizando o dataset

dados_do_ativo = dados.DataReader(ativo, start="2020-01-01",end="2022-11-30", data_source='yahoo')
#dados_do_ativo = dados.DataReader(ativo, start="2018-01-01",end="2022-11-30", data_source='yahoo')
#dados_do_ativo = dados.DataReader(ativo, start="2018-01-01",end="2022-11-30", data_source='yahoo')

#dados_do_ativo = dados.DataReader(ativo, start="2022-11-01",end="2022-11-20", data_source='yahoo')

In [ ]:
dados_do_ativo

In [ ]:
dados_do_ativo.shape

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [18]:
### Preparando os dados
normalizando = MinMaxScaler(feature_range=(0,1))  ##Normaliza os dados/valores entre 0 e 1
dados_normalizados = normalizando.fit_transform(dados_do_ativo['Close'].values.reshape(-1,1)) # dados_normalizados recebe a coluna close de dados_do_Ativo

previsao_dias = 60 ## previsao usara 2meses

x_treinar, y_treinar = [], [] ## Valores atuais que a rede ira preve

##percorre o periodo de tempo, separando os valores em x,y | Começa de 60 em 60 entre 2020 e 2022
for x in range(previsao_dias, len(dados_normalizados)):
    x_treinar.append(dados_normalizados[x-previsao_dias:x, 0])
    y_treinar.append(dados_normalizados[x, 0 ])
    
##Coloca os dados em matriz    
x_treinar, y_treinar = np.array(x_treinar), np.array(y_treinar)
x_treinar = np.reshape(x_treinar, (x_treinar.shape[0], x_treinar.shape[1], 1))

In [ ]:
### Construindo o Modelo


modelo = Sequential()

modelo.add(LSTM(units=50, return_sequences=True, input_shape=(x_treinar.shape[1], 1)))#mesmo formato de dados da matriz
modelo.add(Dropout(0.2))#Desliga 20%dos neuronios a cada passada
modelo.add(LSTM(units=50, return_sequences=True))
modelo.add(Dropout(0.2))
modelo.add(LSTM(units=50))
modelo.add(Dropout(0.2))
modelo.add(Dense(units = 1)) #Prevendo o proximo valor da acao

modelo.compile(optimizer = 'adam', loss = 'mean_squared_error')# optimizador Adam e loss Valor medio ao quadrado
modelo.fit(x_treinar, y_treinar, epochs = 25, batch_size = 32) ## 25 interações 32 pacotes d einformação a cada paçada

In [20]:
### Preparar dados teste

teste_inicio = dt.datetime(2020,1,1) # Inicio do teste
teste_final = dt.datetime.now()   #Final do teste

dados_teste = web.DataReader(ativo, 'yahoo', teste_inicio, teste_final)
precos_reais = dados_teste['Close'].values

total_dados = pd.concat((dados_do_ativo['Close'], dados_teste['Close']), axis = 0)  #total_dados = os dados de traino e os dados de teste

modelo_entrada = total_dados[len(total_dados) - len(dados_teste) - previsao_dias:].values #total de dados menos os dados de teste
modelo_entrada = modelo_entrada.reshape(-1, 1)
modelo_entrada = normalizando.transform(modelo_entrada)#normaliza o modelo de entrada

In [22]:
###Previsao nos valores de teste

x_teste = []

##percorre o periodo de tempo, separando os valores em x,y | Começa de 60 em 60 entre 2020 e 2022
for x in range(previsao_dias, len(modelo_entrada)):
    x_teste.append(modelo_entrada[x-previsao_dias:x, 0])
    
    ##Coloca os dados em matriz 
x_teste = np.array(x_teste)
x_teste = np.reshape(x_teste, (x_teste.shape[0], x_teste.shape[1], 1))
 
previsao_precos = modelo.predict(x_teste) #previsao precos = modelo
previsao_precos = normalizando.inverse_transform(previsao_precos)#normaliza os dados 0a1

24/24 [==============================] - 1s 7ms/step


In [ ]:
### Montando a Representação Grafica do Modelo

plt.plot(precos_reais, color ='red', label = f"Valor Real das acoes de {ativo}")
plt.plot(previsao_precos, color="green", label = f"Previsao das acoes de {ativo}" )
plt.title(f"{ativo} Preco Acao")
plt.xlabel('Tempo')
plt.ylabel(f"{ativo} Preco Acao")
plt.legend()
plt.show()

In [ ]:
###Prevendo Valores Futuros

dados_reais =  [modelo_entrada[len(modelo_entrada) + 1 - previsao_dias:len(modelo_entrada + 1), 0]]
dados_reais =  np.array(dados_reais)
dados_reais = np.reshape(dados_reais, (dados_reais.shape[0], dados_reais.shape[1], 1))

previsao = modelo.predict(dados_reais)
previsao = normalizando.inverse_transform(previsao)

print(f"Previsao para amanha: {previsao}")